In [1]:
import torch
import torch.nn as nn
from transformers import ElectraTokenizer, ElectraForSequenceClassification
import pandas as pd
from tqdm import tqdm 

In [2]:
# koELECTRA 토크나이저 불러오기
tokenizer = ElectraTokenizer.from_pretrained("koelectra-base-v3-discriminator")

In [3]:
''' 모델가중치를 불러오거나 체크포인트를 불러올 때 실행 '''

# 모델의 상태 딕셔너리를 로드합니다.
model_state_dict = torch.load("Ternary_model_state_dict_learning_New_epoch_15.pt")

# 모델을 생성하고 상태를 로드합니다.
model = ElectraForSequenceClassification.from_pretrained("koelectra-base-v3-discriminator", num_labels=3)
model.load_state_dict(model_state_dict)

# 옵티마이저의 상태 딕셔너리를 로드합니다.
optimizer_state_dict = torch.load("Ternary_optimizer_state_learning_dict_New_epoch_15.pt")

Some weights of the model checkpoint at koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense

In [4]:
# 변경하고자 하는 Dropout 비율
new_dropout_rate = 0.2

# 모든 Dropout 레이어의 비율 변경
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Dropout):
        module.p = new_dropout_rate


In [ ]:
# 모델 구조 확인
print(model)

In [ ]:
# 장치 설정 (GPU 사용을 위해)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

## 실제로 예측해보기

In [6]:
# 한국어 문장을 입력으로 받아서 예측 라벨을 출력하는 함수
def predict_label(sentence, model, tokenizer, device):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()
        return predicted_label

In [7]:
df = pd.read_csv("../dataset/2021산림복지_본문_맞춤법검사.txt", sep="\t", encoding='UTF-8')
df

,split_str,org_idx,correct_str
0,세종시청 홈페이지 공공근로 알바 세종시 고용센터 일자리지원센터 교차로 구인구직 사진...,0,세종시청 홈페이지 공공근로 알바 세종시 고용센터 일자리 지원센터 교차로 구인·구직 ...
1,최근에는 스마트시티 미래차 모빌리티 바이오헬스 스마트 그린 융합부품 소재산업 등 세...,0,최근에는 스마트시티 미래 차 모빌리티 생명 건강 스마트 그린 융합부품 소재산업 등 ...
2,사진출처 세종시청 홈페이지 전반적으로 고용률은 높아지고 있지만 어르신 장애인 경력단...,0,사진 출처 세종시청 홈페이지 전반적으로 고용률은 높아지고 있지만 어르신 장애인 경력...
3,사진출처 세종시 교차로 구인구직 세종재가노인지원센터 세종시니어클럽 세종재가노인지원센...,0,사진 출처 세종시 교차로 구인·구직 세종 재가 노인지원센터 세종시니어클럽 세종 재가...
4,세종시청 홈페이지 공공근로등 공공일자리 사업 참여자 모집 공고 확인 및 참여방법 세...,0,세종시청 홈페이지 공공근로 등 공공일자리 사업 참여자 모집 공고 확인 및 참여 방법...
...,...,...,...
222284,이날 전달한 목제품은 편백나무로 만든 도마와 소나무로 만든 칼꽂이다,8431,이날 전달한 목제품은 편백나무로 만든 도마와 소나무로 만든 칼 꽂히다
222285,제작에 사용된 재료는 관내에서 생산한 목재를 활용했다,8431,제작에 사용된 재료는 관내에서 생산한 목재를 활용했다
222286,한편 시 관계자는 풍부한 산림자원을 활용해 실용성 높은 목제품을 만들고 어려운 이웃...,8431,한편 시 관계자는 풍부한 산림자원을 활용해 실용성 높은 목제품을 만들고 어려운 이웃...
222287,며 앞으로 더욱 다양한 목공활동을 통해 시민들이 목제품에 더욱 친근하게 다가설 수 ...,8431,며 앞으로 더욱 다양한 목공활동을 통해 시민들이 목제품에 더욱 친근하게 다가설 수 ...


In [8]:
# 한국어 문장 입력 받기
korean_sentences = df['correct_str'].tolist()

# 예측 라벨 출력
emotion_labels = {0:'부정', 1:'중립', 2:'긍정'}
predicted_label = [emotion_labels[predict_label(korean_sentence, model, tokenizer, device)] for korean_sentence in tqdm(korean_sentences)]
df['label'] = predicted_label

100%|██████████| 222289/222289 [2:57:22<00:00, 20.89it/s]  


In [9]:
ddff = df[['correct_str','label','org_idx']]
ddff

,correct_str,label,org_idx
0,세종시청 홈페이지 공공근로 알바 세종시 고용센터 일자리 지원센터 교차로 구인·구직 ...,긍정,0
1,최근에는 스마트시티 미래 차 모빌리티 생명 건강 스마트 그린 융합부품 소재산업 등 ...,긍정,0
2,사진 출처 세종시청 홈페이지 전반적으로 고용률은 높아지고 있지만 어르신 장애인 경력...,긍정,0
3,사진 출처 세종시 교차로 구인·구직 세종 재가 노인지원센터 세종시니어클럽 세종 재가...,긍정,0
4,세종시청 홈페이지 공공근로 등 공공일자리 사업 참여자 모집 공고 확인 및 참여 방법...,긍정,0
...,...,...,...
222284,이날 전달한 목제품은 편백나무로 만든 도마와 소나무로 만든 칼 꽂히다,부정,8431
222285,제작에 사용된 재료는 관내에서 생산한 목재를 활용했다,긍정,8431
222286,한편 시 관계자는 풍부한 산림자원을 활용해 실용성 높은 목제품을 만들고 어려운 이웃...,긍정,8431
222287,며 앞으로 더욱 다양한 목공활동을 통해 시민들이 목제품에 더욱 친근하게 다가설 수 ...,긍정,8431


In [10]:
ddff['label'].value_counts()

긍정    181662
부정     40439
중립       188
Name: label, dtype: int64

In [11]:
ddff.rename(columns={'correct_str': 'sentence'}, inplace=True)
ddff.rename(columns={'label': 'predicted_sentiment'}, inplace=True)
ddff.to_csv("./Ternary_classification_2021.csv", index=False, encoding='UTF-8')

c:\Users\user\anaconda3\envs\NIFOS\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
